## Ternary Search

In [2]:
import numpy as np
import math


def attempting_insertion_using_ternary_search(alpha,delta,n,m,user_samples):
    """
    Perform the localization phase of DAME using a private ternary search.

    This procedure partitions the interval [-1, 1] into three segments at each iteration,
    uses randomized response to privately count how many user sample-means fall into the
    left and right thirds, and discards the less-likely segment. It repeats until all groups
    are exhausted, returning the remaining interval.

    Parameters:
    ----------
    alpha : float
        Differential privacy parameter for the binary (randomized response) queries.
    delta : float
        Failure probability tolerated in the localization phase.
    n : int
        Number of users allocated to localization (should be n/2 of the total).
    m : int
        Number of samples per user.
    user_samples : list of array-like
        A list of length n, each entry an array-like of m real-valued samples for one user.

    Returns:
    -------
    List :
        [L,R] = The estimated interval that might contain true mean where L is the  lower and R is the upper bound.
    """
    # Precomputing the probability of truthful response under randomized response
    pi_alpha = np.exp(alpha) / (1 + np.exp(alpha))

    # Computing group size b_max to balance accuracy and privacy
    denom = np.log(2 * m / math.log(12)) - 2 * math.log(3 / 2)
    b1 = (2 * n * np.log(3 / 2)) / denom
    b2 = (2 / ((2 * pi_alpha - 1) ** 2)) * np.log(2 * n / delta)
    b_max = int(math.ceil(max(b1, b2)))

    # Maximum number of ternary-search rounds
    t_max = n // b_max

    # Initializing search interval [L, R]
    L, R = -1.0, 1.0
    p = pi_alpha

    # Performing t_max iterations of private ternary search
    for t in range(t_max):
        # Length of each third
        gamma = (R - L) / 3.0
        I1_L, I1_R = L, L + gamma  # left interval
        I3_L, I3_R = R - gamma, R  # right interval

        # Initialize noisy counts for left and right intervals
        V1_tilde = 0
        V3_tilde = 0

        # Process each user in the current group of size b_max
        start = t * b_max
        end = start + b_max
        for x in user_samples[start:end]:
            x_bar = np.mean(x)  # sample-mean for this user

            # Determine true membership in I1 and I3
            V1 = 1 if (I1_L <= x_bar <= I1_R) else 0
            V3 = 1 if (I3_L <= x_bar <= I3_R) else 0

            # Apply randomized response: truth with prob p, flip with prob 1-p
            if np.random.rand() < p:
                V1_tilde += V1
                V3_tilde += V3
            else:
                V1_tilde += 1 - V1
                V3_tilde += 1 - V3

        # Discard the interval (I1 or I3) with smaller noisy count
        if V1_tilde < V3_tilde:
            # Drop the left third: shift L to the start of I2
            L += gamma
        else:
            # Drop the right third: shift R to the end of I2
            R -= gamma

    # Final estimate of the interval containing true mean
    return [L,R]



## DAME with Ternary Search

In [4]:
def dame_with_ternary_search(n, alpha, m, user_samples):
    """
    Implements DAME algorithm with ternary search localization and Laplace estimation.

    Args:
        n: number of users (integer, even)
        alpha: privacy parameter
        m: number of samples per user
        user_samples: list or array of shape (n, m)
    Returns:
        bar_theta: aggregated estimator
    """
    # Compute tau and delta
    tau = (2 * math.log(max(8 * math.sqrt(m * n) * (alpha ** 2), 1))) / m
    pi_alpha = math.exp(alpha) / (1 + math.exp(alpha))
    delta = 2 * n * math.exp(-n * (2 * pi_alpha - 1)**2 / 2)

    # Localization phase
    # use first half of users for localization
    X1 = user_samples[:int(n/2)]


    # loc_samples = user_samples[:n//2]
    theta_hat_loc = attempting_insertion_using_ternary_search(alpha, delta, n//2, m, X1)

    # Estimation phase using second half
    X2 = user_samples[int(n/2):]
    # est_samples = user_samples[n//2:]

    hat_thetas = []
    scale = 14 * tau / alpha
    for x in X2:
        x_bar = np.mean(x)
        if x_bar < theta_hat_loc[0]:
            x_bar = theta_hat_loc[0]
        if x_bar > theta_hat_loc[1]:
            x_bar = theta_hat_loc[1]
        noisy = x_bar + scale * np.random.laplace(0, 1)
        hat_thetas.append(noisy)

    # Aggregation
    bar_theta = (2 / n) * sum(hat_thetas)
    return bar_theta




In [19]:
np.random.seed(42)
n = 100  # total users
m = 50   # samples per user
alpha = 0.5  # privacy parameter
true_mean = 0.3

# Generate user_samples with true_mean plus noise
user_samples = [np.random.normal(loc=true_mean, scale=0.5, size=m) for _ in range(n)]



In [20]:
print(len(user_samples))

100


In [21]:
# Run the algorithm
bar_theta_estimate = dame_with_ternary_search(n, alpha, m, user_samples)

print(bar_theta_estimate)

0.3241274254265732
